In [ ]:
!pip install transformers-interpret
!pip install -U sentence-transformers
!pip3 install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 455.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer

import requests
from bs4 import BeautifulSoup

import re

1. **find a website to scrape text from. Feel free to use Beautiful Soup or another library of your choosing**

In [ ]:
URL = "https://blogs.nvidia.com/blog/what-is-a-transformer-model/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

2. **Clean up the text - if you use a different website this part may be different**

In [ ]:
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
visible_text = soup.getText()
visible_text = ' '.join(re.split(r'[ \n\t]+',visible_text))

3. **Initialize your tokenizer and pretrained Industry Classifer**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

4. **Print out the taxonomy**

In [ ]:
model.config.id2label

{0: 'Advertising',
 1: 'Aerospace & Defense',
 2: 'Apparel Retail',
 3: 'Apparel, Accessories & Luxury Goods',
 4: 'Application Software',
 5: 'Asset Management & Custody Banks',
 6: 'Auto Parts & Equipment',
 7: 'Biotechnology',
 8: 'Building Products',
 9: 'Casinos & Gaming',
 10: 'Commodity Chemicals',
 11: 'Communications Equipment',
 12: 'Construction & Engineering',
 13: 'Construction Machinery & Heavy Trucks',
 14: 'Consumer Finance',
 15: 'Data Processing & Outsourced Services',
 16: 'Diversified Metals & Mining',
 17: 'Diversified Support Services',
 18: 'Electric Utilities',
 19: 'Electrical Components & Equipment',
 20: 'Electronic Equipment & Instruments',
 21: 'Environmental & Facilities Services',
 22: 'Gold',
 23: 'Health Care Equipment',
 24: 'Health Care Facilities',
 25: 'Health Care Services',
 26: 'Health Care Supplies',
 27: 'Health Care Technology',
 28: 'Homebuilding',
 29: 'Hotels, Resorts & Cruise Lines',
 30: 'Human Resource & Employment Services',
 31: 'IT Co

5. **Create your explainer with the model and tokenizer**

In [ ]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

6. **Get local prediction and explanation (note that the max lenngth for a input text is 512)**

In [ ]:
word_attributions = multiclass_explainer(text='many AI engineers are working on trillion-parameter transformers ')

In [ ]:
##prediction
multiclass_explainer.predicted_class_name

'Electrical Components & Equipment'

7. **Print all scores for the each token relative to your local prediction**

In [ ]:
word_attributions

[('[CLS]', 0.0),
 ('many', 0.03520081036633706),
 ('ai', 0.20328935202544038),
 ('engineers', 0.36764973518819655),
 ('are', 0.13188759945640402),
 ('working', -0.01932507246160825),
 ('on', -0.008266055212605155),
 ('trillion', 0.1290773207772646),
 ('-', 0.307560860774914),
 ('parameter', 0.23356465313733926),
 ('transformers', 0.7991400406180841),
 ('[SEP]', 0.0)]

8. **Print out the visualization - can you interpret it?**

In [ ]:
html = multiclass_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
19,Electrical Components & Equipment (0.98),Electrical Components & Equipment,2.18,[CLS] many ai engineers are working on trillion - parameter transformers [SEP]


9. **what happens if you specify a class, such as "Restaurants", how do the weights change? **

In [ ]:
word_attributions = multiclass_explainer(text='i love going to baseball games. yankee fan for life. aaron judge is my fav player', class_name="Movies & Entertainment")

10. **Look at the visualization**

In [ ]:
html = multiclass_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
40,Movies & Entertainment (0.83),Movies & Entertainment,2.63,[CLS] i love going to baseball games . yankee fan for life . aaron judge is my fa ##v player [SEP]
